In [ ]:
import scanpy as sc
import numpy as np
import pandas as pd
from spatialdata import read_zarr

In [ ]:
sdata = read_zarr('/media/duan/DuanLab_Data/openFISH/Decode/Data/20250522_RelnWT/Result_WT/raw_sdata.zarr')
sdata

In [ ]:
areas = []
for pg in sdata.shapes["MultiModel"]["geometry"]:
    areas.append(pg.area)

In [ ]:
adata = sdata.tables['table'].copy()

In [ ]:
adata.obs['area'] = areas
adata.obs

In [ ]:
NP_Pad = pd.read_csv("/media/duan/DuanLab_Data/openFISH/ABA110_Demo/NP_Fold_V58.csv", index_col=0)

In [ ]:
def rotate_coordinates(coords, angle):
    theta = np.radians(angle)
    rotation_matrix = np.array([[np.cos(theta), -np.sin(theta)],
                               [np.sin(theta), np.cos(theta)]])
    coords_array = np.array(coords)
    rotated_coords = coords_array @ rotation_matrix.T
    return rotated_coords

In [ ]:
adata.obsm['spatial'] = adata.obs[['x', 'y']].to_numpy()
adata.obsm["spatial"][:,1] =  0 - adata.obsm["spatial"][:,1]

In [ ]:
sc.set_figure_params(figsize = (12,6), dpi = 100)
sc.pl.embedding(adata, basis = 'spatial', color = 'area', size = 10)

In [ ]:
tmpmatrix = adata.to_df().copy()

In [ ]:
tmpmatrix = tmpmatrix.astype('float32')

In [ ]:
import numpy as np

for col in tmpmatrix.columns:
    # tmpmatrix.loc[:,col] = tmpmatrix.loc[:,col] * np.log2(NP_Pad.loc[col, 'Fold'] + 1)
    tmpmatrix.loc[:,col] = tmpmatrix.loc[:,col] * NP_Pad.loc[col, 'Fold']

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
adata.layers['counts'] = adata.X.copy()
adata.X = csr_matrix(tmpmatrix)
adata.layers['counts_corrected'] = adata.X.copy()

In [ ]:
adata

In [ ]:
adata.var["FP"] = adata.var_names.str.startswith("sFP")

In [ ]:
sc.pp.calculate_qc_metrics(
    adata, inplace=True, log1p=True, percent_top=None,qc_vars=['FP']
)

In [ ]:
sc.pl.violin(
    adata,
    ["n_genes_by_counts", "total_counts", 'pct_counts_FP'],
    jitter=0.4,
    multi_panel=True,
)

In [ ]:
adata.obs['total_counts'].mean()

In [ ]:
adata.obs['total_counts'].median()

In [ ]:
np.max(adata.obs['total_counts_FP'])

In [ ]:
adata.obs['total_counts_FP'].mean()

In [ ]:
adata.obs['pct_counts_FP'].mean()

In [ ]:
np.percentile(adata.obs["pct_counts_FP"], 99)

In [ ]:
# Filter cells according to Cell Volume
adata.obs["area"].mean()

In [ ]:
np.percentile(adata.obs["area"], 1)

In [ ]:
np.percentile(adata.obs["area"], 90)

In [ ]:
np.percentile(adata.obs["area"], 50)

In [ ]:
adata = adata[(adata.obs["area"] > 600) & (adata.obs["area"] < (3 * adata.obs["area"].mean())),:].copy()
adata

In [ ]:
adata.obs['total_counts'].median()

In [ ]:
adata = adata[adata.obs.pct_counts_FP < 10, :].copy()
sc.pp.filter_cells(adata, min_genes=2)
sc.pp.filter_cells(adata, max_counts=1000)
sc.pp.filter_cells(adata, min_counts=10)

In [ ]:
adata

In [ ]:
adata.var_names

In [ ]:
adata.obs['total_counts'].median()

In [ ]:
BLACK_LIST = ['sFP']

In [ ]:
adata = adata[:, [x for x in adata.var_names if x not in BLACK_LIST]].copy()

In [ ]:
adata

In [ ]:
# Normalize using cell area
from scipy.sparse import csr_matrix
cell_area = np.array(adata.obs["area"])
# cell_area = cell_area.reshape((len(cell_area),1))
adata.X = csr_matrix((adata.X.T / cell_area).T)

In [ ]:
# Normalizing to library size
sc.pp.normalize_total(adata, target_sum=109)
# Logarithmize the data
sc.pp.log1p(adata)

In [ ]:
sc.pl.embedding(adata, basis="spatial", color="Slc17a7", size = 5)

In [ ]:
sc.tl.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)

In [ ]:
# Using the igraph implementation and a fixed number of iterations can be significantly faster, especially for larger datasets
sc.tl.leiden(adata, resolution=2, flavor="igraph",n_iterations=-1)

In [ ]:
sc.pl.umap(adata, color = 'leiden')

In [ ]:
sc.pl.embedding(adata, basis="spatial", color="leiden", size = 5)

In [ ]:
adata

In [ ]:
adata.layers['norm'] = adata.X.copy()

In [ ]:
adata.write_h5ad("./RelnWT_filtered.h5ad")

In [ ]:
sdata = read_zarr('/media/duan/DuanLab_Data/openFISH/Decode/Data/20250522_RelnKO/Result_KO/raw_sdata.zarr')
sdata

In [ ]:
areas = []
for pg in sdata.shapes["MultiModel"]["geometry"]:
    areas.append(pg.area)

In [ ]:
adata = sdata.tables['table'].copy()

In [ ]:
adata.obs['area'] = areas
adata.obs

In [ ]:
NP_Pad = pd.read_csv("/media/duan/DuanLab_Data/openFISH/ABA110_Demo/NP_Fold_V58.csv", index_col=0)

In [ ]:
def rotate_coordinates(coords, angle):
    theta = np.radians(angle)
    rotation_matrix = np.array([[np.cos(theta), -np.sin(theta)],
                               [np.sin(theta), np.cos(theta)]])
    coords_array = np.array(coords)
    rotated_coords = coords_array @ rotation_matrix.T
    return rotated_coords

In [ ]:
adata.obsm['spatial'] = adata.obs[['x', 'y']].to_numpy()
adata.obsm["spatial"][:,1] =  0 - adata.obsm["spatial"][:,1]

In [ ]:
sc.set_figure_params(figsize = (10,6), dpi = 100)
sc.pl.embedding(adata, basis = 'spatial', color = 'area', size = 10)

In [ ]:
tmpmatrix = adata.to_df().copy()

In [ ]:
tmpmatrix = tmpmatrix.astype('float32')

In [ ]:
import numpy as np

for col in tmpmatrix.columns:
    # tmpmatrix.loc[:,col] = tmpmatrix.loc[:,col] * np.log2(NP_Pad.loc[col, 'Fold'] + 1)
    tmpmatrix.loc[:,col] = tmpmatrix.loc[:,col] * NP_Pad.loc[col, 'Fold']

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
adata.layers['counts'] = adata.X.copy()
adata.X = csr_matrix(tmpmatrix)
adata.layers['counts_corrected'] = adata.X.copy()

In [ ]:
adata

In [ ]:
adata.var["FP"] = adata.var_names.str.startswith("sFP")

In [ ]:
sc.pp.calculate_qc_metrics(
    adata, inplace=True, log1p=True, percent_top=None,qc_vars=['FP']
)

In [ ]:
sc.pl.violin(
    adata,
    ["n_genes_by_counts", "total_counts", 'pct_counts_FP'],
    jitter=0.4,
    multi_panel=True,
)

In [ ]:
adata.obs['total_counts'].mean()

In [ ]:
adata.obs['total_counts'].median()

In [ ]:
np.max(adata.obs['total_counts_FP'])

In [ ]:
adata.obs['total_counts_FP'].mean()

In [ ]:
# Filter cells according to Cell Volume
adata.obs["area"].mean()

In [ ]:
np.percentile(adata.obs["area"], 1)

In [ ]:
np.percentile(adata.obs["area"], 90)

In [ ]:
np.percentile(adata.obs["area"], 50)

In [ ]:
adata = adata[(adata.obs["area"] > 600) & (adata.obs["area"] < (3 * adata.obs["area"].mean())),:].copy()
adata

In [ ]:
adata.obs['total_counts'].median()

In [ ]:
adata = adata[adata.obs.pct_counts_FP < 10, :].copy()
sc.pp.filter_cells(adata, min_genes=3)
sc.pp.filter_cells(adata, max_counts=1000)
sc.pp.filter_cells(adata, min_counts=10)

In [ ]:
adata

In [ ]:
adata.var_names

In [ ]:
adata.obs['total_counts'].median()

In [ ]:
BLACK_LIST = ['sFP']

In [ ]:
adata = adata[:, [x for x in adata.var_names if x not in BLACK_LIST]].copy()

In [ ]:
adata

In [ ]:
# Normalize using cell area
from scipy.sparse import csr_matrix
cell_area = np.array(adata.obs["area"])
# cell_area = cell_area.reshape((len(cell_area),1))
adata.X = csr_matrix((adata.X.T / cell_area).T)

In [ ]:
# Normalizing to library size
sc.pp.normalize_total(adata, target_sum=109)
# Logarithmize the data
sc.pp.log1p(adata)

In [ ]:
sc.pl.embedding(adata, basis="spatial", color="Slc17a7", size = 5)

In [ ]:
sc.tl.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)

In [ ]:
# Using the igraph implementation and a fixed number of iterations can be significantly faster, especially for larger datasets
sc.tl.leiden(adata, resolution=2, flavor="igraph",n_iterations=-1)

In [ ]:
sc.pl.umap(adata, color = 'leiden')

In [ ]:
sc.pl.embedding(adata, basis="spatial", color="leiden", size = 7)

In [ ]:
adata

In [ ]:
adata.layers['norm'] = adata.X.copy()

In [ ]:
adata.write_h5ad("./RelnKO_filtered.h5ad")

In [ ]:
import scanpy as sc
import numpy as np
import pandas as pd
from spatialdata import read_zarr

In [ ]:
sdata = read_zarr('/media/duan/DuanLab_Data/openFISH/Reln/Reln_0624/WT/raw_sdata.zarr')
sdata

In [ ]:
areas = []
for pg in sdata.shapes["MultiModel"]["geometry"]:
    areas.append(pg.area)

In [ ]:
adata = sdata.tables['table'].copy()

In [ ]:
adata.obs['area'] = areas
adata.obs

In [ ]:
NP_Pad = pd.read_csv("/media/duan/DuanLab_Data/openFISH/Reln/Reln_0624/NP_Fold_V58.csv", index_col=0)

In [ ]:
def rotate_coordinates(coords, angle):
    theta = np.radians(angle)
    rotation_matrix = np.array([[np.cos(theta), -np.sin(theta)],
                               [np.sin(theta), np.cos(theta)]])
    coords_array = np.array(coords)
    rotated_coords = coords_array @ rotation_matrix.T
    return rotated_coords

In [ ]:
adata.obsm['spatial'] = adata.obs[['x', 'y']].to_numpy()
adata.obsm["spatial"][:,1] =  0 - adata.obsm["spatial"][:,1]

In [ ]:
sc.set_figure_params(figsize = (12,6), dpi = 100)
sc.pl.embedding(adata, basis = 'spatial', color = 'area', size = 10)

In [ ]:
tmpmatrix = adata.to_df().copy()

In [ ]:
tmpmatrix = tmpmatrix.astype('float32')

In [ ]:
import numpy as np

for col in tmpmatrix.columns:
    # tmpmatrix.loc[:,col] = tmpmatrix.loc[:,col] * np.log2(NP_Pad.loc[col, 'Fold'] + 1)
    tmpmatrix.loc[:,col] = tmpmatrix.loc[:,col] * NP_Pad.loc[col, 'Fold']

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
adata.layers['counts'] = adata.X.copy()
adata.X = csr_matrix(tmpmatrix)
adata.layers['counts_corrected'] = adata.X.copy()

In [ ]:
adata

In [ ]:
adata.var["FP"] = adata.var_names.str.startswith("False")

In [ ]:
sc.pp.calculate_qc_metrics(
    adata, inplace=True, log1p=True, percent_top=None,qc_vars=['FP']
)

In [ ]:
sc.pl.violin(
    adata,
    ["n_genes_by_counts", "total_counts", 'pct_counts_FP'],
    jitter=0.4,
    multi_panel=True,
)

In [ ]:
adata.obs['total_counts'].mean()

In [ ]:
adata.obs['total_counts'].median()

In [ ]:
np.max(adata.obs['total_counts_FP'])

In [ ]:
adata.obs['total_counts_FP'].mean()

In [ ]:
adata.obs['pct_counts_FP'].mean()

In [ ]:
# Filter cells according to Cell Volume
adata.obs["area"].mean()

In [ ]:
np.percentile(adata.obs["area"], 1)

In [ ]:
np.percentile(adata.obs["area"], 90)

In [ ]:
np.percentile(adata.obs["area"], 50)

In [ ]:
adata = adata[(adata.obs["area"] > 600) & (adata.obs["area"] < (3 * adata.obs["area"].mean())),:].copy()
adata

In [ ]:
adata.obs['total_counts'].median()

In [ ]:
adata = adata[adata.obs.pct_counts_FP < 10, :].copy()
sc.pp.filter_cells(adata, min_genes=2)
sc.pp.filter_cells(adata, max_counts=800)
sc.pp.filter_cells(adata, min_counts=10)

In [ ]:
adata

In [ ]:
adata.var_names

In [ ]:
adata.obs['total_counts'].median()

In [ ]:
BLACK_LIST = ['FalsePositive', 'Dlk1']

In [ ]:
adata = adata[:, [x for x in adata.var_names if x not in BLACK_LIST]].copy()

In [ ]:
adata

In [ ]:
# Normalize using cell area
from scipy.sparse import csr_matrix
cell_area = np.array(adata.obs["area"])
# cell_area = cell_area.reshape((len(cell_area),1))
adata.X = csr_matrix((adata.X.T / cell_area).T)

In [ ]:
# Normalizing to library size
sc.pp.normalize_total(adata, target_sum=109)
# Logarithmize the data
sc.pp.log1p(adata)

In [ ]:
sc.pl.embedding(adata, basis="spatial", color="Slc17a7", size = 5)

In [ ]:
sc.tl.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)

In [ ]:
# Using the igraph implementation and a fixed number of iterations can be significantly faster, especially for larger datasets
sc.tl.leiden(adata, resolution=2, flavor="igraph",n_iterations=-1)

In [ ]:
sc.pl.umap(adata, color = 'leiden')

In [ ]:
sc.pl.embedding(adata, basis="spatial", color="leiden", size = 5)

In [ ]:
adata

In [ ]:
adata.layers['norm'] = adata.X.copy()

In [ ]:
adata.write_h5ad("./RelnWT_0624_filtered.h5ad")

In [ ]:
sdata = read_zarr('/media/duan/DuanLab_Data/openFISH/Reln/Reln_0624/KO/raw_sdata.zarr')
sdata

In [ ]:
areas = []
for pg in sdata.shapes["MultiModel"]["geometry"]:
    areas.append(pg.area)

In [ ]:
adata = sdata.tables['table'].copy()

In [ ]:
adata.obs['area'] = areas
adata.obs

In [ ]:
NP_Pad = pd.read_csv("/media/duan/DuanLab_Data/openFISH/Reln/Reln_0624/NP_Fold_V58.csv", index_col=0)

In [ ]:
def rotate_coordinates(coords, angle):
    theta = np.radians(angle)
    rotation_matrix = np.array([[np.cos(theta), -np.sin(theta)],
                               [np.sin(theta), np.cos(theta)]])
    coords_array = np.array(coords)
    rotated_coords = coords_array @ rotation_matrix.T
    return rotated_coords

In [ ]:
adata.obsm['spatial'] = adata.obs[['x', 'y']].to_numpy()
adata.obsm["spatial"][:,1] =  0 - adata.obsm["spatial"][:,1]

In [ ]:
sc.set_figure_params(figsize = (10,6), dpi = 100)
sc.pl.embedding(adata, basis = 'spatial', color = 'area', size = 10)

In [ ]:
tmpmatrix = adata.to_df().copy()

In [ ]:
tmpmatrix = tmpmatrix.astype('float32')

In [ ]:
import numpy as np

for col in tmpmatrix.columns:
    # tmpmatrix.loc[:,col] = tmpmatrix.loc[:,col] * np.log2(NP_Pad.loc[col, 'Fold'] + 1)
    tmpmatrix.loc[:,col] = tmpmatrix.loc[:,col] * NP_Pad.loc[col, 'Fold']

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
adata.layers['counts'] = adata.X.copy()
adata.X = csr_matrix(tmpmatrix)
adata.layers['counts_corrected'] = adata.X.copy()

In [ ]:
adata

In [ ]:
adata.var["FP"] = adata.var_names.str.startswith("False")

In [ ]:
sc.pp.calculate_qc_metrics(
    adata, inplace=True, log1p=True, percent_top=None,qc_vars=['FP']
)

In [ ]:
sc.pl.violin(
    adata,
    ["n_genes_by_counts", "total_counts", 'pct_counts_FP'],
    jitter=0.4,
    multi_panel=True,
)

In [ ]:
adata.obs['total_counts'].mean()

In [ ]:
adata.obs['total_counts'].median()

In [ ]:
np.max(adata.obs['total_counts_FP'])

In [ ]:
adata.obs['total_counts_FP'].mean()

In [ ]:
# Filter cells according to Cell Volume
adata.obs["area"].mean()

In [ ]:
np.percentile(adata.obs["area"], 1)

In [ ]:
np.percentile(adata.obs["area"], 90)

In [ ]:
np.percentile(adata.obs["area"], 50)

In [ ]:
adata = adata[(adata.obs["area"] > 600) & (adata.obs["area"] < (3 * adata.obs["area"].mean())),:].copy()
adata

In [ ]:
adata.obs['total_counts'].median()

In [ ]:
adata = adata[adata.obs.pct_counts_FP < 10, :].copy()
sc.pp.filter_cells(adata, min_genes=3)
sc.pp.filter_cells(adata, max_counts=1000)
sc.pp.filter_cells(adata, min_counts=10)

In [ ]:
adata

In [ ]:
adata.var_names

In [ ]:
adata.obs['total_counts'].median()

In [ ]:
BLACK_LIST = ['FalsePositive', 'Dlk1']

In [ ]:
adata = adata[:, [x for x in adata.var_names if x not in BLACK_LIST]].copy()

In [ ]:
adata

In [ ]:
# Normalize using cell area
from scipy.sparse import csr_matrix
cell_area = np.array(adata.obs["area"])
# cell_area = cell_area.reshape((len(cell_area),1))
adata.X = csr_matrix((adata.X.T / cell_area).T)

In [ ]:
# Normalizing to library size
sc.pp.normalize_total(adata, target_sum=109)
# Logarithmize the data
sc.pp.log1p(adata)

In [ ]:
sc.pl.embedding(adata, basis="spatial", color="Slc17a7", size = 5)

In [ ]:
sc.tl.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)

In [ ]:
# Using the igraph implementation and a fixed number of iterations can be significantly faster, especially for larger datasets
sc.tl.leiden(adata, resolution=2, flavor="igraph",n_iterations=-1)

In [ ]:
sc.pl.umap(adata, color = 'leiden')

In [ ]:
sc.pl.embedding(adata, basis="spatial", color="leiden", size = 7)

In [ ]:
adata

In [ ]:
adata.layers['norm'] = adata.X.copy()

In [ ]:
adata.write_h5ad("./RelnKO_0624_filtered.h5ad")

In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
from read_roi import read_roi_file
from shapely import Polygon,Point

In [ ]:
adata

In [ ]:
sc.pl.embedding(adata, basis = 'spatial', color = 'cell_type')

In [ ]:
roi_CA1 = read_roi_file("/media/duan/DuanLab_Data/openFISH/Reln/Reeler_reAnnotation_ROIs/WT1/CA1.roi")
roi_CA1

In [ ]:
key = 'CA1'
polygon = Polygon(

        np.column_stack([roi_CA1[key]['x'], roi_CA1[key]['y']])

)
polygon

In [ ]:
adata.obs['CA1'] = adata.obs[['x', 'y']].apply(lambda row: polygon.contains(Point([row['x'],row['y']])), axis = 1)

In [ ]:
sc.pl.embedding(adata, basis = 'spatial', color = 'CA1')

In [ ]:
new_cell_type = []
for _, row in adata.obs.iterrows():
    # if row['cell_type'] == 'Ext CA1' and row['CA1'] == True:
    #     new_cell_type.append('Ext CA1')
    if row['cell_type'] == 'Ext CA1' and row['CA1'] == False:
        new_cell_type.append('BACKGROUND')
    else:
        new_cell_type.append(row['cell_type'])
        
adata.obs['cell_type'] = new_cell_type

In [ ]:
sc.pl.embedding(adata, basis = 'spatial', color = 'cell_type', groups = ['Ext CA1'], size = 10)

In [ ]:
roi_CA3 = read_roi_file("/media/duan/DuanLab_Data/openFISH/Reln/Reeler_reAnnotation_ROIs/WT1/CA3.roi")
roi_CA3

In [ ]:
key = 'CA3'
polygon = Polygon(

        np.column_stack([roi_CA3[key]['x'], roi_CA3[key]['y']])

)
polygon

In [ ]:
adata.obs['CA3'] = adata.obs[['x', 'y']].apply(lambda row: polygon.contains(Point([row['x'],row['y']])), axis = 1)

In [ ]:
sc.pl.embedding(adata, basis = 'spatial', color = 'CA3')

In [ ]:
new_cell_type = []
for _, row in adata.obs.iterrows():
    # if row['cell_type'] == 'Ext CA1' and row['CA1'] == True:
    #     new_cell_type.append('Ext CA1')
    if row['cell_type'] == 'Ext CA2' and row['CA3'] == False:
        new_cell_type.append('BACKGROUND')
    elif row['cell_type'] == 'Ext CA2' and row['CA3'] == True:
        new_cell_type.append('Ext CA3')
    else:
        new_cell_type.append(row['cell_type'])
        
adata.obs['cell_type'] = new_cell_type

In [ ]:
sc.pl.embedding(adata, basis = 'spatial', color = 'cell_type', groups = ['Ext CA3'], size = 10)

In [ ]:
roi_CB = read_roi_file("/media/duan/DuanLab_Data/openFISH/Reln/Reeler_reAnnotation_ROIs/WT1/CB.roi")
roi_CB

In [ ]:
key = 'CB'
polygon = Polygon(

        np.column_stack([roi_CB[key]['x'], roi_CB[key]['y']])

)
polygon

In [ ]:
adata.obs['CB'] = adata.obs[['x', 'y']].apply(lambda row: polygon.contains(Point([row['x'],row['y']])), axis = 1)

In [ ]:
sc.pl.embedding(adata, basis = 'spatial', color = 'CB')

In [ ]:
new_cell_type = []
for _, row in adata.obs.iterrows():
    if row['cell_type'].startswith('Astro-') and row['CB'] == True:
        new_cell_type.append('Astro-CB')
    elif row['cell_type'] == 'Ext CB' and row['CB'] == False:
        new_cell_type.append('BACKGROUND')
    elif row['cell_type'] == 'Inh CB' and row['CB'] == False:
        new_cell_type.append('BACKGROUND')
    else:
        new_cell_type.append(row['cell_type'])
        
adata.obs['cell_type'] = new_cell_type

In [ ]:
sc.pl.embedding(adata, basis = 'spatial', color = 'cell_type', groups = ['Astro-CB', 'Inh CB', 'Ext CB'], size = 10)

In [ ]:
roi_DG = read_roi_file("/media/duan/DuanLab_Data/openFISH/Reln/Reeler_reAnnotation_ROIs/WT1/DG.roi")
roi_DG

In [ ]:
key = 'DG'
polygon = Polygon(

        np.column_stack([roi_DG[key]['x'], roi_DG[key]['y']])

)
polygon

In [ ]:
adata.obs['DG'] = adata.obs[['x', 'y']].apply(lambda row: polygon.contains(Point([row['x'],row['y']])), axis = 1)

In [ ]:
sc.pl.embedding(adata, basis = 'spatial', color = 'DG')

In [ ]:
new_cell_type = []
for _, row in adata.obs.iterrows():
    if row['cell_type'] == 'Ext DG' and row['DG'] == False:
        new_cell_type.append('BACKGROUND')
    else:
        new_cell_type.append(row['cell_type'])
        
adata.obs['cell_type'] = new_cell_type

In [ ]:
sc.pl.embedding(adata, basis = 'spatial', color = 'cell_type', groups = ['Ext DG'], size = 10)

In [ ]:
roi_MB = read_roi_file("/media/duan/DuanLab_Data/openFISH/Reln/Reeler_reAnnotation_ROIs/WT1/MB.roi")
roi_MB

In [ ]:
key = 'MB'
polygon = Polygon(

        np.column_stack([roi_MB[key]['x'], roi_MB[key]['y']])

)
polygon

In [ ]:
adata.obs['MB'] = adata.obs[['x', 'y']].apply(lambda row: polygon.contains(Point([row['x'],row['y']])), axis = 1)

In [ ]:
sc.pl.embedding(adata, basis = 'spatial', color = 'MB')

## MB之后再处理

In [ ]:
sc.pl.embedding(adata, basis = 'spatial', color = 'cell_type', groups = ['Ext MB', 'Inh MB'], size = 10)

In [ ]:
sc.pl.embedding(adata, basis = 'spatial', color = 'Hap1', size = 10)

In [ ]:
roi_NT = read_roi_file("/media/duan/DuanLab_Data/openFISH/Reln/Reeler_reAnnotation_ROIs/WT1/NT.roi")
roi_NT

In [ ]:
key = 'NT'
polygon = Polygon(

        np.column_stack([roi_NT[key]['x'], roi_NT[key]['y']])

)
polygon

In [ ]:
adata.obs['NT'] = adata.obs[['x', 'y']].apply(lambda row: polygon.contains(Point([row['x'],row['y']])), axis = 1)

In [ ]:
sc.pl.embedding(adata, basis = 'spatial', color = 'NT')

In [ ]:
new_cell_type = []
for _, row in adata.obs.iterrows():
    if row['cell_type'].startswith('Astro-') and row['NT'] == True:
        new_cell_type.append('Astro-NT')
    else:
        new_cell_type.append(row['cell_type'])
        
adata.obs['cell_type'] = new_cell_type

In [ ]:
sc.pl.embedding(adata, basis = 'spatial', color = 'cell_type', groups = ['Astro-NT', 'Astro-TE'], size = 10)

In [ ]:
roi_TE = read_roi_file("/media/duan/DuanLab_Data/openFISH/Reln/Reeler_reAnnotation_ROIs/WT1/TE.roi")
roi_TE

In [ ]:
key = 'TE'
polygon = Polygon(

        np.column_stack([roi_TE[key]['x'], roi_TE[key]['y']])

)
polygon

In [ ]:
adata.obs['TE'] = adata.obs[['x', 'y']].apply(lambda row: polygon.contains(Point([row['x'],row['y']])), axis = 1)

In [ ]:
sc.pl.embedding(adata, basis = 'spatial', color = 'TE')

In [ ]:
new_cell_type = []
for _, row in adata.obs.iterrows():
    if row['cell_type'].startswith('Astro-') and row['TE'] == True:
        new_cell_type.append('Astro-TE')
    else:
        new_cell_type.append(row['cell_type'])
        
adata.obs['cell_type'] = new_cell_type

In [ ]:
sc.pl.embedding(adata, basis = 'spatial', color = 'cell_type', groups = ['Astro-NT', 'Astro-TE', 'Astro-CB'], size = 10)

In [ ]:
roi_MCB = read_roi_file("/media/duan/DuanLab_Data/openFISH/Reln/Reeler_reAnnotation_ROIs/WT1/MEA-COA-BMA.roi")
roi_MCB

In [ ]:
key = 'MEA-COA-BMA'
polygon = Polygon(

        np.column_stack([roi_MCB[key]['x'], roi_MCB[key]['y']])

)
polygon

In [ ]:
adata.obs['MEA-COA-BMA'] = adata.obs[['x', 'y']].apply(lambda row: polygon.contains(Point([row['x'],row['y']])), axis = 1)

In [ ]:
sc.pl.embedding(adata, basis = 'spatial', color = 'MEA-COA-BMA')

In [ ]:
adata.write_h5ad("RelnWT_Manul.h5ad")

In [ ]:
adata = sc.read_h5ad("RelnWT_Manul.h5ad")

In [ ]:
df = pd.read_csv("/media/duan/DuanLab_Data/openFISH/Reln/Reeler_reAnnotation_ROIs/WT1/TH.csv", skiprows=2)
df

In [ ]:
polygon = Polygon(df[df['Selection'] == 'Selection 1'][['X', 'Y']].to_numpy())

In [ ]:
adata.obs['TH'] = adata.obs[['x', 'y']].apply(lambda row: polygon.contains(Point([row['x']*0.325,row['y']*0.325])), axis = 1)

In [ ]:
sc.pl.embedding(adata, basis = 'spatial', color = 'TH')

In [ ]:
df = pd.read_csv("/media/duan/DuanLab_Data/openFISH/Reln/Reeler_reAnnotation_ROIs/WT1/MB.csv", skiprows=2)
df

In [ ]:
polygon = Polygon(df[df['Selection'] == 'Selection 1'][['X', 'Y']].to_numpy())

In [ ]:
adata.obs['MB'] = adata.obs[['x', 'y']].apply(lambda row: polygon.contains(Point([row['x']*0.325,row['y']*0.325])), axis = 1)

In [ ]:
sc.pl.embedding(adata, basis = 'spatial', color = 'MB')

In [ ]:
adata.write_h5ad("RelnWT_Manul.h5ad")

In [ ]:
adata = sc.read_h5ad("RelnWT_Manul.h5ad")

In [ ]:
df = pd.read_csv("/media/duan/DuanLab_Data/openFISH/Reln/Reeler_reAnnotation_ROIs/WT1/IC.csv", skiprows=2)
df

In [ ]:
polygon = Polygon(df[df['Selection'] == 'Selection 1'][['X', 'Y']].to_numpy())

In [ ]:
adata.obs['IC'] = adata.obs[['x', 'y']].apply(lambda row: polygon.contains(Point([row['x']*0.325,row['y']*0.325])), axis = 1)

In [ ]:
sc.pl.embedding(adata, basis = 'spatial', color = 'IC')

In [ ]:
df = pd.read_csv("/media/duan/DuanLab_Data/openFISH/Reln/Reeler_reAnnotation_ROIs/WT1/STR.csv", skiprows=2)
df

In [ ]:
polygon = Polygon(df[df['Selection'] == 'Selection 1'][['X', 'Y']].to_numpy())

In [ ]:
adata.obs['STR'] = adata.obs[['x', 'y']].apply(lambda row: polygon.contains(Point([row['x']*0.325,row['y']*0.325])), axis = 1)

In [ ]:
sc.pl.embedding(adata, basis = 'spatial', color = 'STR')

In [ ]:
adata.write_h5ad("RelnWT_Manul.h5ad")

In [ ]:
import scanpy as sc
from shapely import Polygon, Point
import numpy as np
from read_roi import read_roi_file

In [ ]:
adata = sc.read_h5ad("RelnAll_Annotated.h5ad")
adata

In [ ]:
sc.pl.embedding(adata, basis = 'spatial', color = 'cell_type')

In [ ]:
roi = read_roi_file('/media/duan/DuanLab_Data/openFISH/Reln/ROIs_Cortex/KO0522_cortex_ridge.roi')
roi

In [ ]:
key = 'KO0522_cortex_ridge'
polygon = Polygon(

        np.column_stack([roi[key]['x'], roi[key]['y']])

)
polygon

In [ ]:
adata1 = adata[adata.obs['Sample'] == 'KO1'].copy()

In [ ]:
adata1.obs['Cortex_Ridge'] = adata1.obs[['x', 'y']].apply(lambda row: polygon.contains(Point([row['x'],row['y']])), axis = 1)

In [ ]:
adata1 = adata1[adata1.obs['Cortex_Ridge']].copy()

In [ ]:
sc.pl.embedding(adata1, basis = 'spatial_raw', color = 'Cortex_Ridge')

In [ ]:
def rotate_coordinates(coords, angle):
    theta = np.radians(angle)
    rotation_matrix = np.array([[np.cos(theta), -np.sin(theta)],
                               [np.sin(theta), np.cos(theta)]])
    coords_array = np.array(coords)
    rotated_coords = coords_array @ rotation_matrix.T
    return rotated_coords

In [ ]:
sc.set_figure_params(figsize=(3,7), scanpy = False)
adata1.obsm['spatial_rotate'] = rotate_coordinates(adata1.obsm['spatial_raw'], 29)
sc.pl.embedding(adata1, basis = 'spatial_rotate', color = 'Cortex_Ridge')

In [ ]:
sc.pl.embedding(adata1, basis = 'spatial_rotate', color = 'cell_type', save = 'KO1_cortex_ridge.pdf')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(7, 3))

sns.kdeplot(adata1[adata1.obs['cell_type'] == 'Ext L2/3'].obsm['spatial_rotate'][:,1], label='Ext L2/3', color = '#E9530F')
sns.kdeplot(adata1[adata1.obs['cell_type'] == 'Ext L4/5'].obsm['spatial_rotate'][:,1], label='Ext L4/5', color = '#3DCCB7')
sns.kdeplot(adata1[adata1.obs['cell_type'] == 'Ext L6'].obsm['spatial_rotate'][:,1], label='Ext L6', color = '#34661F')

plt.legend()

plt.savefig('KO1_cortex_kde_plot.pdf', dpi=300)  # 可以调整dpi和bbox_inches参数

plt.show()

In [ ]:
roi = read_roi_file('/media/duan/DuanLab_Data/openFISH/Reln/ROIs_Cortex/KO0624_cortex_ridge.roi')
roi

In [ ]:
key = 'KO0624_cortex_ridge'
polygon = Polygon(

        np.column_stack([roi[key]['x'], roi[key]['y']])

)
polygon

In [ ]:
adata1 = adata[adata.obs['Sample'] == 'KO2'].copy()

In [ ]:
adata1.obs['Cortex_Ridge'] = adata1.obs[['x', 'y']].apply(lambda row: polygon.contains(Point([row['x'],row['y']])), axis = 1)

In [ ]:
adata1 = adata1[adata1.obs['Cortex_Ridge']].copy()

In [ ]:
sc.pl.embedding(adata1, basis = 'spatial_raw', color = 'Cortex_Ridge')

In [ ]:
def rotate_coordinates(coords, angle):
    theta = np.radians(angle)
    rotation_matrix = np.array([[np.cos(theta), -np.sin(theta)],
                               [np.sin(theta), np.cos(theta)]])
    coords_array = np.array(coords)
    rotated_coords = coords_array @ rotation_matrix.T
    return rotated_coords

In [ ]:
sc.set_figure_params(figsize=(3,7), scanpy = False)
adata1.obsm['spatial_rotate'] = rotate_coordinates(adata1.obsm['spatial_raw'], 29)
sc.pl.embedding(adata1, basis = 'spatial_rotate', color = 'Cortex_Ridge')

In [ ]:
sc.pl.embedding(adata1, basis = 'spatial_rotate', color = 'cell_type',  save = 'KO2_cortex_ridge.pdf')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(7, 3))

sns.kdeplot(adata1[adata1.obs['cell_type'] == 'Ext L2/3'].obsm['spatial_rotate'][:,1], label='Ext L2/3', color = '#E9530F')
sns.kdeplot(adata1[adata1.obs['cell_type'] == 'Ext L4/5'].obsm['spatial_rotate'][:,1], label='Ext L4/5', color = '#3DCCB7')
sns.kdeplot(adata1[adata1.obs['cell_type'] == 'Ext L6'].obsm['spatial_rotate'][:,1], label='Ext L6', color = '#34661F')

plt.legend()

plt.savefig('KO2_cortex_kde_plot.pdf', dpi=300)  # 可以调整dpi和bbox_inches参数

plt.show()

In [ ]:
roi = read_roi_file('/media/duan/DuanLab_Data/openFISH/Reln/ROIs_Cortex/WT0522_cortex_ridge.roi')
roi

In [ ]:
key = 'WT0522_cortex_ridge'
polygon = Polygon(

        np.column_stack([roi[key]['x'], roi[key]['y']])

)
polygon

In [ ]:
adata1 = adata[adata.obs['Sample'] == 'WT1'].copy()

In [ ]:
adata1.obs['Cortex_Ridge'] = adata1.obs[['x', 'y']].apply(lambda row: polygon.contains(Point([row['x'],row['y']])), axis = 1)

In [ ]:
adata1 = adata1[adata1.obs['Cortex_Ridge']].copy()

In [ ]:
sc.pl.embedding(adata1, basis = 'spatial_raw', color = 'Cortex_Ridge')

In [ ]:
def rotate_coordinates(coords, angle):
    theta = np.radians(angle)
    rotation_matrix = np.array([[np.cos(theta), -np.sin(theta)],
                               [np.sin(theta), np.cos(theta)]])
    coords_array = np.array(coords)
    rotated_coords = coords_array @ rotation_matrix.T
    return rotated_coords

In [ ]:
sc.set_figure_params(figsize=(3,7), scanpy = False)
adata1.obsm['spatial_rotate'] = rotate_coordinates(adata1.obsm['spatial_raw'], 29)
sc.pl.embedding(adata1, basis = 'spatial_rotate', color = 'Cortex_Ridge')

In [ ]:
sc.pl.embedding(adata1, basis = 'spatial_rotate', color = 'cell_type',  save = 'WT1_cortex_ridge.pdf')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(7, 3))

sns.kdeplot(adata1[adata1.obs['cell_type'] == 'Ext L2/3'].obsm['spatial_rotate'][:,1], label='Ext L2/3', color = '#E9530F')
sns.kdeplot(adata1[adata1.obs['cell_type'] == 'Ext L4/5'].obsm['spatial_rotate'][:,1], label='Ext L4/5', color = '#3DCCB7')
sns.kdeplot(adata1[adata1.obs['cell_type'] == 'Ext L6'].obsm['spatial_rotate'][:,1], label='Ext L6', color = '#34661F')

plt.legend()

plt.savefig('WT1_cortex_kde_plot.pdf', dpi=300)  # 可以调整dpi和bbox_inches参数

plt.show()

In [ ]:
roi = read_roi_file('/media/duan/DuanLab_Data/openFISH/Reln/ROIs_Cortex/WT0624_cortex_ridge.roi')
roi

In [ ]:
key = 'WT0624_cortex_ridge'
polygon = Polygon(

        np.column_stack([roi[key]['x'], roi[key]['y']])

)
polygon

In [ ]:
adata1 = adata[adata.obs['Sample'] == 'WT2'].copy()

In [ ]:
adata1.obs['Cortex_Ridge'] = adata1.obs[['x', 'y']].apply(lambda row: polygon.contains(Point([row['x'],row['y']])), axis = 1)

In [ ]:
adata1 = adata1[adata1.obs['Cortex_Ridge']].copy()

In [ ]:
sc.pl.embedding(adata1, basis = 'spatial_raw', color = 'Cortex_Ridge')

In [ ]:
def rotate_coordinates(coords, angle):
    theta = np.radians(angle)
    rotation_matrix = np.array([[np.cos(theta), -np.sin(theta)],
                               [np.sin(theta), np.cos(theta)]])
    coords_array = np.array(coords)
    rotated_coords = coords_array @ rotation_matrix.T
    return rotated_coords

In [ ]:
sc.set_figure_params(figsize=(3,7), scanpy = False)
adata1.obsm['spatial_rotate'] = rotate_coordinates(adata1.obsm['spatial_raw'], 29)
sc.pl.embedding(adata1, basis = 'spatial_rotate', color = 'Cortex_Ridge')

In [ ]:
sc.pl.embedding(adata1, basis = 'spatial_rotate', color = 'cell_type',  save = 'WT2_cortex_ridge.pdf')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(7, 3))

sns.kdeplot(adata1[adata1.obs['cell_type'] == 'Ext L2/3'].obsm['spatial_rotate'][:,1], label='Ext L2/3', color = '#E9530F')
sns.kdeplot(adata1[adata1.obs['cell_type'] == 'Ext L4/5'].obsm['spatial_rotate'][:,1], label='Ext L4/5', color = '#3DCCB7')
sns.kdeplot(adata1[adata1.obs['cell_type'] == 'Ext L6'].obsm['spatial_rotate'][:,1], label='Ext L6', color = '#34661F')

plt.legend()

plt.savefig('WT2_cortex_kde_plot.pdf', dpi=300)  # 可以调整dpi和bbox_inches参数

plt.show()